# Service Creation
## Load Config

In [128]:
import json
import os
from azure.core.exceptions import ResourceNotFoundError 

In [129]:
PROJECT_PATH = "/Users/vinay/Projects/my-mlops/"

In [130]:
def load_app_config():
    config_path = os.path.join(PROJECT_PATH, "config", "config.json")
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)

cfg = load_app_config()

In [131]:
RG_CONFIG = cfg.get("resource_group", {})
RESOURCE_GROUP = RG_CONFIG.get("name")
LOCATION = RG_CONFIG.get("location", "eastus")

VIRTUAL_MACHINE_CFG = cfg.get("vm", {})
VM_NAME = VIRTUAL_MACHINE_CFG.get("name")
VM_SIZE = VIRTUAL_MACHINE_CFG.get("size")
ADMIN_USERNAME = VIRTUAL_MACHINE_CFG.get("admin_username")
SSH_KEY_PATH = VIRTUAL_MACHINE_CFG.get("ssh_public_key_path")
VIRTUAL_NETWORK = VIRTUAL_MACHINE_CFG.get("virtual_network")
SUB_NETWORK = VIRTUAL_MACHINE_CFG.get("sub_network")
NETWORK_SECURITY_GRP = VIRTUAL_MACHINE_CFG.get("network_security_grp")
PUBLIC_IP = VIRTUAL_MACHINE_CFG.get("public_ip")
NETWORK_INTERFACE = VIRTUAL_MACHINE_CFG.get("network_interface")

STORAGE_CFG = cfg.get("storage", {})
STORAGE_ACCOUNT_NAME = STORAGE_CFG.get("storage_account")
CONTAINERS = STORAGE_CFG.get("containers")

# loading subscription id
with open(os.path.join(PROJECT_PATH,"tokens","subscription_id"), "r") as f:
    SUBSCRIPTION_ID = f.read()

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.storage import StorageManagementClient

def get_resource_client(subscription_id, service):
    """Authenticate and return the ResourceManagementClient"""

    credential = DefaultAzureCredential()
    if service == 'RC':
        return ResourceManagementClient(credential, subscription_id)
    if service == 'NC':
        return NetworkManagementClient(credential, subscription_id)
    if service == 'CC':
        return ComputeManagementClient(credential, subscription_id)
    if service == 'SA':
        return StorageManagementClient(credential, subscription_id)

In [133]:
RESOURCE_CLIENT = get_resource_client(SUBSCRIPTION_ID, 'RC')

## Resource Group

In [ ]:
def validate_resource_group(resource_client, resource_grp):
    """Check if a resource group exists"""
    print(f"🔍 Validating resource group '{resource_grp}'...")
    exists = resource_client.resource_groups.check_existence(resource_grp)
    if exists:
        print(f"✅ Resource group '{resource_grp}' exists.")
    else:
        print(f"❌ Resource group '{resource_grp}' does not exist.")
    return exists

##### Validating if resource group is available if not create one

In [135]:
if validate_resource_group(RESOURCE_CLIENT, RESOURCE_GROUP):
    print(f"ℹ️ Resource group '{RESOURCE_GROUP}' already exists.")
else:
    print(f"🚀 Creating resource group '{RESOURCE_GROUP}' in '{LOCATION}'...")
    rg_result = RESOURCE_CLIENT.resource_groups.create_or_update(
        RESOURCE_GROUP, {"location": LOCATION}
    )
    print(f"✅ Resource group '{rg_result.name}' created successfully.")

🔍 Validating resource group 'rg-mymlops'...
✅ Resource group 'rg-mymlops' exists.
ℹ️ Resource group 'rg-mymlops' already exists.


## Virtual machine

In [136]:
from azure.mgmt.compute.models import (
    HardwareProfile, OSProfile, LinuxConfiguration,
    SshConfiguration, SshPublicKey, NetworkInterfaceReference,
    StorageProfile, ImageReference, OSDisk, DiskCreateOptionTypes
)
from azure.mgmt.network.models import (
    NetworkSecurityGroup, SecurityRule, PublicIPAddress,
    NetworkInterface, Subnet, VirtualNetwork, IPConfiguration
)

In [137]:
print(f"🚀 Loading network client")
NETWORK_CLIENT = get_resource_client(SUBSCRIPTION_ID, 'NC')

🚀 Loading network client


##### Virtual network setup

In [138]:
print("🔹 Checking Virtual Network...")

try:
    # Try to get existing VNet
    vnet = NETWORK_CLIENT.virtual_networks.get(
        RESOURCE_GROUP,
        VIRTUAL_NETWORK
    )
    print(f"✅ Virtual Network '{vnet.name}' already exists. Using existing configuration.")

except ResourceNotFoundError:
    # Create only if not found
    print("⚙️ Virtual Network not found. Creating new one...")
    vnet = NETWORK_CLIENT.virtual_networks.begin_create_or_update(
        RESOURCE_GROUP,
        VIRTUAL_NETWORK,
        {
            "location": LOCATION,
            "address_space": {"address_prefixes": ["10.0.0.0/16"]}
        },
    ).result()
    print(f"✅ Virtual Network '{vnet.name}' created successfully.")

🔹 Checking Virtual Network...
✅ Virtual Network 'vnet-mlops' already exists. Using existing configuration.


##### Subnet setup

In [141]:
print("🔹 Checking Subnet...")

try:
    # Try to get existing subnet
    subnet = NETWORK_CLIENT.subnets.get(
        RESOURCE_GROUP,
        VIRTUAL_NETWORK,
        SUB_NETWORK
    )
    print(f"✅ Subnet '{subnet.name}' already exists. Using existing configuration.")

except ResourceNotFoundError:
    # Create only if not found
    print("⚙️ Subnet not found. Creating new one...")
    subnet = NETWORK_CLIENT.subnets.begin_create_or_update(
        RESOURCE_GROUP,
        VIRTUAL_NETWORK,
        SUB_NETWORK,
        {"address_prefix": "10.0.0.0/24"},
    ).result()
    print(f"✅ Subnet '{subnet.name}' created successfully.")

🔹 Checking Subnet...
✅ Subnet 'subnet-mlops' already exists. Using existing configuration.


##### Network Security group setup

In [142]:
print("🔹 Checking Network Security Group (NSG)...")

try:
    # Try to get existing NSG
    nsg = NETWORK_CLIENT.network_security_groups.get(
        RESOURCE_GROUP,
        NETWORK_SECURITY_GRP
    )
    print(f"✅ NSG '{nsg.name}' already exists. Using existing configuration.")

except ResourceNotFoundError:
    # Create only if not found
    print("⚙️ NSG not found. Creating new one (Allow SSH)...")
    nsg = NETWORK_CLIENT.network_security_groups.begin_create_or_update(
        RESOURCE_GROUP,
        NETWORK_SECURITY_GRP,
        {
            "location": LOCATION,
            "security_rules": [
                SecurityRule(
                    name="AllowSSH",
                    protocol="Tcp",
                    direction="Inbound",
                    access="Allow",
                    priority=1000,
                    source_address_prefix="*",
                    destination_address_prefix="*",
                    source_port_range="*",
                    destination_port_range="22",
                )
            ],
        },
    ).result()
    print(f"✅ NSG '{nsg.name}' created successfully.")

🔹 Checking Network Security Group (NSG)...
✅ NSG 'nsg-mlops' already exists. Using existing configuration.


##### Public IP setup

In [143]:
print("🔹 Checking Public IP...")

try:
    # Try to get existing Public IP
    public_ip = NETWORK_CLIENT.public_ip_addresses.get(
        RESOURCE_GROUP,
        PUBLIC_IP
    )
    print(f"✅ Public IP '{public_ip.name}' already exists. Using existing configuration.")
    if public_ip.ip_address:
        print(f"🌐 Current IP Address: {public_ip.ip_address}")

except ResourceNotFoundError:
    # Create only if not found
    print("⚙️ Public IP not found. Creating new one...")
    public_ip = NETWORK_CLIENT.public_ip_addresses.begin_create_or_update(
        RESOURCE_GROUP,
        PUBLIC_IP,
        {
            "location": LOCATION,
            "public_ip_allocation_method": "Dynamic"
        },
    ).result()
    print(f"✅ Public IP '{public_ip.name}' created successfully.")
    if public_ip.ip_address:
        print(f"🌐 Assigned IP Address: {public_ip.ip_address}")

🔹 Checking Public IP...
✅ Public IP 'pip-mlops' already exists. Using existing configuration.


In [144]:
print("🔹 Checking Network Interface...")

try:
    # Try to get existing NIC
    nic = NETWORK_CLIENT.network_interfaces.get(
        RESOURCE_GROUP,
        NETWORK_INTERFACE
    )
    print(f"✅ Network Interface '{nic.name}' already exists. Using existing configuration.")

except ResourceNotFoundError:
    # Create only if not found
    print("⚙️ Network Interface not found. Creating new one...")
    nic = NETWORK_CLIENT.network_interfaces.begin_create_or_update(
        RESOURCE_GROUP,
        NETWORK_INTERFACE,
        {
            "location": LOCATION,
            "ip_configurations": [
                IPConfiguration(
                    name="ipconfig1",
                    subnet=Subnet(id=subnet.id),
                    public_ip_address=PublicIPAddress(id=public_ip.id),
                    network_security_group=NetworkSecurityGroup(id=nsg.id),
                )
            ],
        },
    ).result()
    print(f"✅ Network Interface '{nic.name}' created successfully.")

🔹 Checking Network Interface...
✅ Network Interface 'nic-mlops' already exists. Using existing configuration.


##### Virtual machine setup

In [ ]:
# VM Configuration
print("🔹 Defining VM configuration...")
with open(SSH_KEY_PATH, "r") as f:
    ssh_key_data = f.read().strip()

vm_parameters = {
    "location": LOCATION,
    "hardware_profile": HardwareProfile(vm_size=VM_SIZE),
    "storage_profile": StorageProfile(
        image_reference=ImageReference(
            publisher="Canonical",
            offer="UbuntuServer",
            sku="18.04-LTS",
            version="latest",
        ),
        os_disk=OSDisk(
            create_option=DiskCreateOptionTypes.from_image,
            managed_disk={"storage_account_type": "StandardSSD_LRS"},
        ),
    ),
    "os_profile": OSProfile(
        computer_name=VM_NAME,
        admin_username=ADMIN_USERNAME,
        linux_configuration=LinuxConfiguration(
            disable_password_authentication=True,
            ssh=SshConfiguration(
                public_keys=[
                    SshPublicKey(
                        path=f"/home/{ADMIN_USERNAME}/.ssh/authorized_keys",
                        key_data=ssh_key_data,
                    )
                ]
            ),
        ),
    ),
    "network_profile": {
        "network_interfaces": [NetworkInterfaceReference(id=nic.id)]},
}

🔹 Defining VM configuration...


In [ ]:
print(f"🚀 Loading compute client")
COMPUTE_CLIENT = get_resource_client(SUBSCRIPTION_ID, 'CC')

In [ ]:
print(f"🔹 Checking if VM '{VM_NAME}' already exists...")

try:
    # Try to get existing VM
    vm_result = COMPUTE_CLIENT.virtual_machines.get(
        RESOURCE_GROUP,
        VM_NAME
    )
    print(f"✅ VM '{vm_result.name}' already exists. Using existing configuration.")

except ResourceNotFoundError:
    # Create only if not found
    print(f"⚙️ VM '{VM_NAME}' not found. Creating new one... this may take a few minutes.")
    creation = COMPUTE_CLIENT.virtual_machines.begin_create_or_update(
        RESOURCE_GROUP,
        VM_NAME,
        vm_parameters
    )
    vm_result = creation.result()
    print(f"✅ VM '{vm_result.name}' created successfully.")

🚀 Creating VM 'vm-mlops' ... this may take a few minutes.


##### Attaching NSG to NIC

In [ ]:
# Get NIC attached to the VM
nic_id = vm_result.network_profile.network_interfaces[0].id.split('/')[-1]

# Fetch NIC details
nic = NETWORK_CLIENT.network_interfaces.get(RESOURCE_GROUP, nic_id)

# Check if NSG is attached directly to NIC
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = NETWORK_CLIENT.network_security_groups.get(RESOURCE_GROUP, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} \
              {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22


In [28]:
# Attach NSG to NIC
nic.network_security_group = nsg

In [ ]:
# Update NIC with the NSG association
update_nic = NETWORK_CLIENT.network_interfaces.begin_create_or_update(
    RESOURCE_GROUP,
    nic_id,
    nic
)

In [31]:
result = update_nic.result()
print(f"✅ NSG '{nsg.name}' successfully attached to NIC '{nic_id}'.")

✅ NSG 'nsg-mlops' successfully attached to NIC 'nic-mlops'.


In [ ]:
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = NETWORK_CLIENT.network_security_groups.get(RESOURCE_GROUP, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22


## Blob Storage

In [ ]:
credential = DefaultAzureCredential()


In [112]:
credential = DefaultAzureCredential()
storage_client = StorageManagementClient(credential, SUBSCRIPTION_ID)

In [ ]:
# Create Blob Storage with hierarchical namespace (Data Lake Gen2)
storage_async_op = storage_client.storage_accounts.begin_create(
    RESOURCE_GROUP,
    STORAGE_ACCOUNT_NAME,
    {
        "location": LOCATION,
        "kind": "StorageV2",
        "sku": {"name": "Standard_LRS"},
        "is_hns_enabled": True  # Hierarchical namespace enabled
    },
)
storage_account = storage_async_op.result()
print(f"✅ Storage account '{storage_account.name}' created.")

✅ Storage account 'storagemymlops' created.


In [121]:
storage_client = StorageManagementClient(credential, SUBSCRIPTION_ID)

In [123]:
# 🔹 Fetch the existing storage account details
storage_account = storage_client.storage_accounts.get_properties(
    resource_group_name=RESOURCE_GROUP,
    account_name=STORAGE_ACCOUNT_NAME
)

print(f"✅ Loaded existing storage account: {storage_account.name}")
print(f"📍 Location: {storage_account.location}")
print(f"📦 Kind: {storage_account.kind}")

ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [119]:
# ==== Get Storage Account Key ====
print("🔑 Retrieving access key...")
keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP, STORAGE_ACCOUNT_NAME)
storage_key = keys.keys[0].value

🔑 Retrieving access key...


ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [ ]:
# ==== Save Key to File ====
key_file_path = os.path.join(PROJECT_PATH, "tokens", STORAGE_ACCOUNT_NAME)
print(key_file_path)
with open(key_file_path, "w") as key_file:
    key_file.write(storage_key)
print(f"🗝️ Storage account key saved to: {key_file_path}")

/Users/vinay/Projects/my-mlops/tokens/storagemymlops
🗝️ Storage account key saved to: /Users/vinay/Projects/my-mlops/tokens/storagemymlops


In [ ]:
# ==== Get Storage Account Key ====
print("🔑 Retrieving access key...")
keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP, STORAGE_ACCOUNT_NAME)
storage_key = keys.keys[0].value

🔑 Retrieving access key...


NameError: name 'RESOURCE_GROUP' is not defined

In [ ]:
# ==== Connect to Blob Service ====
from azure.storage.blob import BlobServiceClient
blob_service_client = BlobServiceClient(
    f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
    credential=storage_key)

In [ ]:
# ==== Create Containers ====
for container in CONTAINERS.split(','):
    try:
        blob_service_client.create_container(container)
        print(f"✅ Container '{container}' created successfully.")
    except Exception as e:
        if "ContainerAlreadyExists" in str(e):
            print(f"⚠️ Container '{container}' already exists.")
        else:
            print(f"❌ Failed to create container '{container}': {e}")

⚠️ Container 'bronze' already exists.
⚠️ Container 'silver' already exists.
⚠️ Container 'gold' already exists.


### Uploading find to S3